In [1]:
import pandas as pd

In [37]:
feature_abt_buy_test = pd.read_csv("abt_buy/features_abt_buy_test")
feature_abt_buy_train = pd.read_csv("abt_buy/features_abt_buy_train")
abt_buy_gold_standard_train = pd.read_csv("abt_buy/abt_buy_gold_standard_train.csv",sep=";")
abt_buy_gold_standard_test = pd.read_csv("abt_buy/abt_buy_gold_standard_test.csv",sep=";")

feature_amazon_google_test = pd.read_csv("amazon_google/features_amazon_google_test")
feature_amazon_google_train = pd.read_csv("amazon_google/features_amazon_google_train")

feature_wdc_phones_test = pd.read_csv("wdc_product/features_phones_phones_catalog_test")
feature_wdc_phones_train = pd.read_csv("wdc_product/features_phones_phones_catalog_train")

feature_wdc_headphones_test = pd.read_csv("wdc_product/features_headphones_headphones_catalog_test")
feature_wdc_headphones_train = pd.read_csv("wdc_product/features_headphones_headphones_catalog_train")

#feature_author_test = pd.read_csv("author/features_author_test")
#feature_author_train = pd.read_csv("author/features_author_train")

In [29]:
print(feature_abt_buy_train.shape)
print(feature_abt_buy_test.shape)

(5730, 19)
(1432, 19)


In [30]:
print(feature_amazon_google_train.shape)
print(feature_amazon_google_test.shape)

(6753, 24)
(1687, 24)


In [31]:
print(feature_wdc_phones_train.shape)
print(feature_wdc_phones_test.shape)

(1762, 135)
(440, 135)


In [32]:
print(feature_wdc_headphones_train.shape)
print(feature_wdc_headphones_test.shape)

(1163, 142)
(290, 142)


# Preprocessing abt_buy

In [38]:
ids_abt_buy_train = feature_abt_buy_train[["source_id","target_id"]]
ids_abt_buy_test = feature_abt_buy_test[["source_id","target_id"]]
abt_original = pd.read_csv("abt_buy/abt.csv",encoding='latin1').add_prefix('left_')
buy_original = pd.read_csv("abt_buy/buy.csv",encoding='latin1').add_prefix('right_')

#train
abt_buy_train = ids_abt_buy_train.merge(
    abt_original,left_on="source_id",right_on="left_subject_id").merge(
    buy_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
abt_buy_train['id'] = abt_buy_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
abt_buy_test = ids_abt_buy_test.merge(
    abt_original,left_on="source_id",right_on="left_subject_id").merge(
    buy_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
abt_buy_test['id'] = abt_buy_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

In [39]:
#train
abt_buy_gold_standard_train['id'] = abt_buy_gold_standard_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
abt_buy_gold_standard_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
abt_buy_gold_standard_test['id'] = abt_buy_gold_standard_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
abt_buy_gold_standard_test.drop(['source_id','target_id'],axis=1,inplace=True)

In [40]:
abt_buy_train = abt_buy_train.merge(abt_buy_gold_standard_train,on='id')
abt_buy_train = abt_buy_train.rename(columns={'matching':'label'})
abt_buy_train.drop('right_manufacturer',axis=1,inplace=True)
abt_buy_train['label'] = abt_buy_train['label'].astype(int)

In [41]:
abt_buy_test = abt_buy_test.merge(abt_buy_gold_standard_test,on='id')
abt_buy_test = abt_buy_test.rename(columns={'matching':'label'})
abt_buy_test.drop('right_manufacturer',axis=1,inplace=True)
abt_buy_test['label'] = abt_buy_test['label'].astype(int)
abt_buy_test.head()

,source_id,target_id,left_name,left_description,left_price,right_name,right_description,right_price,id,label
0,17187,202041204,Universal IR/RF Remote - MX350,Universal IR/RF Remote - MX350/ Controls Up To...,149.95,Universal MX-350 Osiris Remote Control - MX350,"TV, VCR, DVD Player, CD Player, Cable Box, Sat...",93.9,17187-202041204,1
1,17187,203069939,Universal IR/RF Remote - MX350,Universal IR/RF Remote - MX350/ Controls Up To...,149.95,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,17187-203069939,0
2,33965,203069939,Logitech Harmony One Advanced Universal Remote...,Logitech Harmony One Advanced Universal Remote...,249.00,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,33965-203069939,0
3,31273,203069939,Sony Universal Remote Commander Remote Control...,Sony Universal Remote Commander Remote Control...,NaN,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,31273-203069939,0
4,27779,205844283,Sony LCS-CSQ/B Black Soft Carrying Case - LCSCSQB,Sony LCS-CSQ/B Black Soft Carrying Case - LCSC...,NaN,Sony LCS-CSQ Soft Cyber-shot Camera Case - LCS...,Top Loading - Polyamide - Black,NaN,27779-205844283,1


## Split Train into Train and Validation

In [77]:
y = abt_buy_train['label']

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(abt_buy_train, y,
                                                stratify=y, 
                                                test_size=0.25)

In [80]:
abt_buy_train = X_train
abt_buy_validation = X_val

In [81]:
abt_buy_train.to_csv("abt_buy/abt_buy_train",index=False)
abt_buy_validation.to_csv("abt_buy/abt_buy_validation",index=False)
abt_buy_test.to_csv("abt_buy/abt_buy_test",index=False)